In [1]:
import sys
import pandas as pd
import numpy as np
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from ckiptagger import construct_dictionary
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

host = 4
# host = 0

if host == 0:
    path = r'/Users/aron/Documents/GitHub/Perfume/2_NLP'
    dict_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/dict.xlsx'
    data_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/data.xls'
    
elif host == 4:
    path = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/'
    dict_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/dict.xlsx'
    data_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/data.xls'

    
# Codebase ......
path_codebase = [r'/Users/aron/Documents/GitHub/Arsenal/',
                 r'/home/aronhack/stock_predict/Function',
                 r'D:\GitHub\Arsenal',
                 r'D:\Data_Mining\Projects\Codebase_YZ',
                 r'/home/jupyter/Arsenal/20220522',
                 path + '/Function']


for i in path_codebase:    
    if i not in sys.path:
        sys.path = [i] + sys.path
    

import codebase_yz as cbyz
    
    
path_resource = path + '/Resource'
path_export = path + '/Export'

## Import Custom Dictionary

In [2]:
dict_df = pd.read_excel(dict_file)
dict_df = cbyz.df_lower(df=dict_df, cols=[])
dict_li = dict_df['word'].tolist()
custom_dict = dict((el, 1) for el in dict_li)
custom_dict = construct_dictionary(custom_dict)

In [3]:
# 2. Load model
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
# https://drive.google.com/drive/folders/105IKCb88evUyLKlLondvDBoh7Dy_I1tm

ws = WS(path_resource + "/data")
pos = POS(path_resource + "/data")
ner = NER(path_resource + "/data")

/opt/anaconda3/lib/python3.7/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/opt/anaconda3/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/opt/anaconda3/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
/opt/anaconda3/lib/python3.7/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future ver

## 4. Run the WS-POS-NER pipeline

In [4]:
data = pd.read_excel(data_file)

# Preprocessing
data = data.dropna(subset=['title'], axis=0)
data = data[~data['title'].str.contains('廣告')]
data = cbyz.df_lower(df=data, cols=['title', 'brand'])
data['title'] = data['title'].apply(cbyz.unicode_filter)

# There are bug in the crawler, so clean title by limiting length of string.
data = data[(data['title'].str.len()>=5) & (data['title'].str.len()<50)]

# data = data[~data['name'].isna()]
# data = data[data.index<=300]

data = data.dropna(subset=['title'], axis=0)
data['title'] = data['title'].str.replace('找相似', '')

print('Bug - title裡面還是有括號')

Bug - title裡面還是有括號


In [22]:
sentence_list = data['title'].tolist()

y = 'name'
y = 'brand'
y = 'type'

y_li = data[y].tolist()
# y_data = data[y].tolist()
# y_data = data[y].tolist()

In [17]:
y

'type'

In [ ]:
sentence_list

In [12]:
word_sentence_list = ws(
    sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = custom_dict, # words in this dictionary are forced
)

pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [13]:
word_sentence_list

[['已', '開封', 'clausporto', '10', 'ml', '5', '入', '香水', '組', '350'],
 ['近', '全新', 'clausportocologne', '古龍水', '30', 'ml', '4', '入組', '1680'],
 ['bvlgari',
  '寶格麗',
  '系列',
  '香水',
  '5',
  'ml',
  '男香',
  '女香',
  '現貨',
  '蝦皮',
  '直',
  '送',
  '429',
  '已',
  '售出',
  '433',
  '桃園市',
  '楊梅區'],
 ['bvlgari',
  '寶格麗',
  '晶澈',
  '女性',
  '淡香水',
  'vivo',
  '薇朵',
  '1599',
  '已',
  '售出',
  '26',
  '臺北市',
  '大安區'],
 ['bvlgari',
  '寶格麗',
  '森林',
  '之',
  '光',
  '男性',
  '淡香精',
  '15',
  'ml',
  '可噴式',
  '試管',
  '香水',
  '全新',
  '119',
  '已',
  '售出',
  '7',
  '臺',
  '中市',
  '北區'],
 ['59',
  '折',
  'bvlgari',
  '粉晶',
  '女性',
  '淡香水',
  '65',
  'ml',
  '85',
  '折',
  '37502199',
  '已',
  '售出',
  '1',
  '臺北市',
  '內湖區'],
 ['7',
  '折',
  'bvlgari',
  '大吉嶺',
  '男性',
  '淡香水',
  '50',
  'ml',
  '85',
  '折',
  '28751999',
  '已',
  '售出',
  '4',
  '臺北市',
  '內湖區'],
 ['2022',
  '寶格麗',
  '歡沁',
  '玫香',
  '女性',
  '淡香水',
  '5',
  'ml',
  '典藏瓶',
  '299',
  '已',
  '售出',
  '3',
  '臺南市',
  '北區'],
 ['bvlgari',
  '寶格麗',
 

In [67]:
print('Check - 是否需要為了不同的y調整這一行，type可能需要用equal')

df = pd.DataFrame()
for i, sentence in enumerate(sentence_list):
    
    cur_sentence = word_sentence_list[i]
    cur_y = y_li[i]
    
        
    if cur_y != cur_y:
        # cur_y為na
        cur_tag = [np.nan for k in range(len(cur_sentence))]
    else:
        # cur_y不是na
        cur_tag = []
        for j in cur_sentence:

            if y == 'type':
                if j == cur_y:
                    cur_tag.append(1)
                else:
                    cur_tag.append(0)
            else:
                if j in cur_y:
                    cur_tag.append(1)
                else:
                    cur_tag.append(0)
                
    new_li = list(zip(cur_sentence,  pos_sentence_list[i], cur_tag))
    temp_df = pd.DataFrame(new_li, columns=['word', 'pos', 'tag'])
    temp_df['sentence'] = i
    
    df = df.append(temp_df)
    
df = df[['sentence', 'word', 'pos', 'tag']]
df = cbyz.df_conv_col_type(df=df, cols='tag', to='str')
df

Check - 是否需要為了不同的y調整這一行，type可能需要用equal


,sentence,word,pos,tag
0,0,已,D,nan
1,0,開封,VB,nan
2,0,clausporto,FW,nan
3,0,10,Neu,nan
4,0,ml,Nf,nan
...,...,...,...,...
9,827,已,D,nan
10,827,售出,VC,nan
11,827,1,Neu,nan
12,827,臺中市,Nc,nan


In [68]:
# 6. Show Results
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return


In [69]:
for i, sentence in enumerate(sentence_list):
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)

'已開封clausporto10ml5入香水組350'
已(D)　開封(VB)　clausporto(FW)　10(Neu)　ml(Nf)　5(Neu)　入(VCL)　香水(Na)　組(Na)　350(Neu)　
(1, 3, 'GPE', '開封')
'近全新clausportocologne古龍水30ml4入組1680'
近(VJ)　全新(VH)　clausportocologne(FW)　古龍水(Na)　30(Neu)　ml(Nf)　4(Neu)　入組(VA)　1680(Neu)　
'bvlgari寶格麗系列香水5ml男香女香現貨蝦皮直送429已售出433桃園市楊梅區'
bvlgari(FW)　寶格麗(Nb)　系列(Na)　香水(Na)　5(Neu)　ml(Nf)　男香(Na)　女香(Na)　現貨(Na)　蝦皮(Na)　直(D)　送(VD)　429(Neu)　已(D)　售出(VC)　433(Neu)　桃園市(Nc)　楊梅區(Nc)　
(7, 10, 'ORG', '寶格麗')
(14, 17, 'PRODUCT', '5ml')
(27, 30, 'CARDINAL', '429')
(33, 42, 'GPE', '433桃園市楊梅區')
'bvlgari寶格麗晶澈女性淡香水vivo薇朵1599已售出26臺北市大安區'
bvlgari(FW)　寶格麗(Nb)　晶澈(VH)　女性(Na)　淡香水(Na)　vivo(FW)　薇朵(Nb)　1599(Neu)　已(D)　售出(VC)　26(Neu)　臺北市(Nc)　大安區(Nc)　
(23, 27, 'DATE', '1599')
(30, 32, 'DATE', '26')
(32, 38, 'GPE', '臺北市大安區')
'bvlgari寶格麗森林之光男性淡香精15ml可噴式試管香水全新119已售出7臺中市北區'
bvlgari(FW)　寶格麗(Nb)　森林(Na)　之(DE)　光(Na)　男性(Na)　淡香精(Na)　15(Neu)　ml(Nf)　可噴式(Na)　試管(Na)　香水(Na)　全新(VH)　119(Neu)　已(D)　售出(VC)　7(Neu)　臺(Nf)　中市(Nc)　北區(Nc)　
(0, 7, 'PERSON', 'bvlgari')
(7, 10, 'ORG', '寶格麗')
(19,

歐舒丹(Nb)　玫瑰花(Na)　園(Nc)　淡香水(Na)　10(Neu)　ml(Nf)　200(Neu)　臺中市(Nc)　北區(Nc)　
(0, 3, 'ORG', '歐舒丹')
(10, 17, 'QUANTITY', '10ml200')
(17, 20, 'GPE', '臺中市')
'專櫃正品loccitane歐舒丹櫻花淡香水75ml超值現貨特價滿額折201490臺北市文山區'
專櫃(Nc)　正品(Na)　loccitane(FW)　歐舒丹(Nb)　櫻花(Na)　淡香水(Na)　75(Neu)　ml(Nf)　超值(VH)　現貨(Na)　特價(Na)　滿額(VH)　折(VC)　201490(Neu)　臺北市(Nc)　文山區(Nc)　
(13, 18, 'PRODUCT', '歐舒丹櫻花')
(21, 25, 'QUANTITY', '75ml')
(34, 40, 'DATE', '201490')
(40, 43, 'GPE', '臺北市')
(43, 46, 'LOC', '文山區')
'歐舒丹馬鞭草淡香水10ml專櫃公司貨350已售出10新北市林口區'
歐舒丹(Nb)　馬(Na)　鞭草(Na)　淡香水(Na)　10(Neu)　ml(Nf)　專櫃(Nc)　公司貨(Na)　350(Neu)　已(D)　售出(VC)　10(Neu)　新北市(Nc)　林口區(Nc)　
(0, 3, 'ORG', '歐舒丹')
(9, 13, 'QUANTITY', '10ml')
(26, 32, 'GPE', '新北市林口區')
'愛妮loccitane歐舒丹果漾馬鞭草淡香水100ml1280高雄市路竹區'
愛妮(Nb)　loccitane(FW)　歐舒丹(Nb)　果漾馬鞭草(Na)　淡香水(Na)　100(Neu)　ml(Nf)　1280(Neu)　高雄市(Nc)　路竹區(Nc)　
(0, 16, 'PERSON', '愛妮loccitane歐舒丹果漾')
'歐舒丹香水(小ml)30100已售出10基隆市中正區'
歐舒丹(Nb)　香水(Na)　((PARENTHESISCATEGORY)　小(VH)　ml(Nf)　)(PARENTHESISCATEGORY)　30100(Neu)　已(D)　售出(VC)　10(Neu)　基隆市(Nc)　中正區(Nc)　
(0, 3, 'ORG

山姆(Nb)　百貨(Na)　jaguar(FW)　積架(Nb)　黑爵(Na)　男性(Na)　淡香水(Na)　100(Neu)　ml(Nf)　579(Neu)　已(D)　售出(VC)　4(Neu)　臺北市(Nc)　大同區(Nc)　
(0, 4, 'ORG', '山姆百貨')
(19, 27, 'QUANTITY', '100ml579')
(31, 37, 'GPE', '臺北市大同區')
'山姆百貨jaguar積架捷豹魅力男性淡香水100ml1299臺北市大同區'
山姆(Nb)　百貨(Na)　jaguar(FW)　積架(Nb)　捷豹(Nb)　魅力(Na)　男性(Na)　淡香水(Na)　100(Neu)　ml(Nf)　1299(Neu)　臺北市(Nc)　大同區(Nc)　
(12, 14, 'PRODUCT', '捷豹')
(21, 26, 'QUANTITY', '100ml')
(30, 33, 'GPE', '臺北市')
'jaguar積架捷豹無限領域男性淡香水的試香紙香水會社50已售出2臺北市士林區'
jaguar(FW)　積架(Nb)　捷豹(Nb)　無限(VH)　領域(Na)　男性(Na)　淡香水(Na)　的(DE)　試(VF)　香紙(Na)　香水(Na)　會社(Nc)　50(Neu)　已(D)　售出(VC)　2(Neu)　臺北市(Nc)　士林區(Nc)　
(0, 6, 'ORG', 'jaguar')
(8, 10, 'PRODUCT', '捷豹')
(27, 29, 'CARDINAL', '50')
(32, 33, 'CARDINAL', '2')
(33, 39, 'GPE', '臺北市士林區')
'calvinkleinckfreesport男性香水100ml1200新北市樹林區'
calvinkleinckfreesport(FW)　男性(Na)　香水(Na)　100(Neu)　ml(Nf)　1200(Neu)　新北市(Nc)　樹林區(Nc)　
(36, 41, 'GPE', '北市樹林區')
'calvinkleinckfree男性淡香水5ml分裝小香試香125已售出646臺中市南屯區'
calvinkleinckfree(FW)　男性(Na)　淡香水(Na)　5(Neu)　ml(Nf)　分裝(VC)　小香(Nb)　試香(VA)　125(Neu

<(FW)　麻吉(Nb)　小舖(Nc)　>abercrombiefitchaf(FW)　真我(Na)　男性(Na)　淡香水(Na)　7901240(Neu)　新北市(Nc)　新莊區(Nc)　
(31, 38, 'DATE', '7901240')
(38, 44, 'GPE', '新北市新莊區')
'af真我女性淡香精50100ml(慶開幕全館免運費)11801780高雄市三民區'
af(FW)　真我(Na)　女性(Na)　淡香精(Na)　50100(Neu)　ml(Nf)　((PARENTHESISCATEGORY)　慶(VC)　開幕(VH)　全(Neqa)　館(Nc)　免(VJ)　運費(Na)　)(PARENTHESISCATEGORY)　11801780(Neu)　高雄市(Nc)　三民區(Nc)　
(9, 16, 'QUANTITY', '50100ml')
(26, 34, 'CARDINAL', '11801780')
(34, 40, 'GPE', '高雄市三民區')
'abercrombiefitchbatchno46100425臺北市中正區'
abercrombiefitchbatchno(FW)　46100425(Neu)　臺北市(Nc)　中正區(Nc)　
(31, 34, 'GPE', '臺北市')
(34, 37, 'GPE', '中正區')
'afabercrombiefitchaffierce猛男香水3000已售出9桃園市中壢區'
afabercrombiefitchaffierce(FW)　猛男(Na)　香水(Na)　3000(Neu)　已(D)　售出(VC)　9(Neu)　桃園市(Nc)　中壢區(Nc)　
(30, 34, 'CARDINAL', '3000')
(37, 44, 'GPE', '9桃園市中壢區')
'abercrombiefitch境男性淡香水針管2ml60100臺中市西屯區'
abercrombiefitch(FW)　境(Na)　男性(Na)　淡香水(Na)　針管(Na)　2(Neu)　ml(Nf)　60100(Neu)　臺中市(Nc)　西屯區(Nc)　
(24, 32, 'CARDINAL', '2ml60100')
(32, 38, 'GPE', '臺中市西屯區')
'abercrombiefitchaf真我夜色

mercedesbenz(FW)　賓士(Nb)　任(VG)　行旅(VCL)　者(Na)　系列(Na)　銀色(Na)　風潮(Na)　男性(Na)　淡香水(Na)　450(Neu)　基隆市(Nc)　中正區(Nc)　
(0, 14, 'PERSON', 'mercedesbenz賓士')
(29, 32, 'CARDINAL', '450')
(32, 35, 'GPE', '基隆市')
'mercedesbenzmangrey賓士輝煌之星男性淡香水100ml1780'
mercedesbenzmangrey(FW)　賓士(Nb)　輝煌(VH)　之(DE)　星(Na)　男性(Na)　淡香水(Na)　100(Neu)　ml(Nf)　1780(Neu)　
'霏霏美妝賓士爵色佳人女性淡香水90ml1150已售出11臺北市北投區'
霏霏(VH)　美妝(VB)　賓士(Nb)　爵色(Na)　佳人(Na)　女性(Na)　淡香水(Na)　90(Neu)　ml(Nf)　1150(Neu)　已(D)　售出(VC)　11(Neu)　臺北市(Nc)　北投區(Nc)　
(15, 23, 'QUANTITY', '90ml1150')
(26, 28, 'DATE', '11')
(28, 34, 'GPE', '臺北市北投區')
'蘋果綠mercedesbenz賓士紳藍爵士男性淡香水100ml980臺北市中正區'
蘋果(Na)　綠(VH)　mercedesbenz(FW)　賓士(Nb)　紳(Na)　藍爵士(Nb)　男性(Na)　淡香水(Na)　100(Neu)　ml(Nf)　980(Neu)　臺北市(Nc)　中正區(Nc)　
'mercedesbenz賓士紳藍爵士男性淡香水100ml1050臺中市南區'
mercedesbenz(FW)　賓士(Nb)　紳(Na)　藍爵士(Nb)　男性(Na)　淡香水(Na)　100(Neu)　ml(Nf)　1050(Neu)　臺中市(Nc)　南區(Nc)　
(0, 12, 'PERSON', 'mercedesbenz')
(15, 18, 'PERSON', '藍爵士')
'mercedesbenzselect賓士帝耀非凡男性淡香水(現貨正品1799屏東縣竹田鄉'
mercedesbenzselect(FW)　賓士(Nb)　帝耀(Nb)　非凡(VH)　男性(Na

法拉利(Na)　ferrari(FW)　氫(Na)　元素(Na)　男性(Na)　淡香水(Na)　800(Neu)　臺中市(Nc)　梧棲區(Nc)　
(18, 21, 'QUANTITY', '800')
(21, 27, 'GPE', '臺中市梧棲區')
'ferrari法拉利岩蘭草男性淡香精50ml全新950臺南市安南區'
ferrari(FW)　法拉利(Na)　岩蘭草(Na)　男性(Na)　淡香精(Na)　50(Neu)　ml(Nf)　全新(VH)　950(Neu)　臺南市(Nc)　安南區(Nc)　
(18, 22, 'QUANTITY', '50ml')
(27, 33, 'GPE', '臺南市安南區')
'redpowerice熱力冰火男性淡香水4ml160高雄市小港區'
redpowerice(FW)　熱力(Na)　冰火(Na)　男性(Na)　淡香水(Na)　4(Neu)　ml(Nf)　160(Neu)　高雄市(Nc)　小港區(Nc)　
(26, 32, 'GPE', '高雄市小港區')
'黑色法拉利男性淡香水小香4ml)299臺中市霧峰區'
黑色(Na)　法拉利(Na)　男性(Na)　淡香水(Na)　小香(Nb)　4(Neu)　ml(Nf)　)(PARENTHESISCATEGORY)　299(Neu)　臺中市(Nc)　霧峰區(Nc)　
(2, 5, 'PRODUCT', '法拉利')
(12, 15, 'QUANTITY', '4ml')
(19, 25, 'GPE', '臺中市霧峰區')
'金萊美ferrarif1經典男性淡香水分裝試管針管25ml5090已售出1彰化縣員林市'
金萊美(Nb)　ferrari(FW)　f1(FW)　經典(Na)　男性(Na)　淡香水(Na)　分裝試管(Na)　針管(Na)　25(Neu)　ml(Nf)　5090(Neu)　已(D)　售出(VC)　1(Neu)　彰化縣(Nc)　員林市(Nc)　
(25, 33, 'CARDINAL', '25ml5090')
(36, 43, 'GPE', '1彰化縣員林市')
'蝦皮優選ferrari512tr法拉利男香小香迷你瓶6ml339新北市汐止區'
蝦皮(Na)　優選(Na)　ferrari(FW)　512(Neu)　tr(FW)　法拉利(Na)　男香(Na)　小香(Nb)　

大衛杜夫(Nb)　runwild(FW)　自然(VH)　之(DE)　境(Na)　女用(A)　試管(Na)　香水(Na)　55(Neu)　高雄市(Nc)　鳳山區(Nc)　
(0, 11, 'ORG', '大衛杜夫runwild')
(23, 29, 'GPE', '高雄市鳳山區')
'davidoffcoolwater冷泉男性淡香水125ml現貨650新北市土城區'
davidoff(FW)　coolwater(FW)　冷泉(Na)　男性(Na)　淡香水(Na)　125(Neu)　ml(Nf)　現貨(Na)　650(Neu)　新北市(Nc)　土城區(Nc)　
(24, 29, 'QUANTITY', '125ml')
(34, 40, 'GPE', '新北市土城區')
'波妮小舖davidoffecho大衛杜夫迴響女性淡香精100ml890新北市新店區'
波妮(Nb)　小舖(Nc)　davidoff(FW)　echo(FW)　大衛杜夫(Nb)　迴響(VH)　女性(Na)　淡香精(Na)　100(Neu)　ml(Nf)　890(Neu)　新北市(Nc)　新店區(Nc)　
(16, 20, 'ORG', '大衛杜夫')
'香水專賣店davidoffcoolwater冷泉男香(5ml)199已售出12'
香水(Na)　專賣店(Nc)　davidoff(FW)　coolwater(FW)　冷泉(Na)　男香(Na)　((PARENTHESISCATEGORY)　5(Neu)　ml(Nf)　)(PARENTHESISCATEGORY)　199(Neu)　已(D)　售出(VC)　12(Neu)　
(37, 39, 'CARDINAL', '12')
'davidoffcoolwater大衛杜夫冷泉男性淡香水5ml199雲林縣林內鄉'
davidoff(FW)　coolwater(FW)　大衛杜夫(Nb)　冷泉(Na)　男性(Na)　淡香水(Na)　5(Neu)　ml(Nf)　199(Neu)　雲林縣(Nc)　林內鄉(Nc)　
(17, 21, 'PRODUCT', '大衛杜夫')
(28, 31, 'QUANTITY', '5ml')
(34, 40, 'GPE', '雲林縣林內鄉')
'davidoff大衛杜夫coolwater冷泉男性淡香水125ml850高雄市三民區'
da

(30, 32, 'CARDINAL', '85')
(35, 38, 'CARDINAL', '236')
(38, 44, 'GPE', '高雄市新興區')
'蝦皮優選gatsby男性淡香水50ml大公主小舖130139已售出32新北市中和區'
蝦皮(Na)　優選(Na)　gatsby(FW)　男性(Na)　淡香水(Na)　50(Neu)　ml(Nf)　大(VH)　公主(Na)　小舖(Nc)　130139(Neu)　已(D)　售出(VC)　32(Neu)　新北市(Nc)　中和區(Nc)　
(15, 19, 'QUANTITY', '50ml')
(24, 30, 'DATE', '130139')
(33, 35, 'CARDINAL', '32')
(35, 41, 'GPE', '新北市中和區')
'dunhilllconabsolute金格淡香精1650屏東縣新埤鄉'
dunhill(FW)　lconabsolute(FW)　金格(Nb)　淡香精(Nb)　1650(Neu)　屏東縣(Nc)　新埤鄉(Nc)　
(0, 19, 'PERSON', 'dunhilllconabsolute')
(22, 24, 'PERSON', '香精')
(28, 34, 'GPE', '屏東縣新埤鄉')
'蝦皮優選dunhill513n北緯513度淡香水2ml針管香水80已售出9新北市永和區'
蝦皮(Na)　優選(Na)　dunhill(FW)　513(Neu)　n(FW)　北緯(Na)　513(Neu)　度(Nf)　淡香水(Na)　2(Neu)　ml(Nf)　針管(Na)　香水(Na)　80(Neu)　已(D)　售出(VC)　9(Neu)　新北市(Nc)　永和區(Nc)　
(24, 27, 'QUANTITY', '2ml')
(31, 33, 'CARDINAL', '80')
(36, 37, 'CARDINAL', '9')
'dunhillicon經典男性淡香精試香40150已售出13桃園市龜山區'
dunhill(FW)　icon(FW)　經典(Na)　男性(Na)　淡香精(Na)　試(VF)　香(Na)　40150(Neu)　已(D)　售出(VC)　13(Neu)　桃園市(Nc)　龜山區(Nc)　
(20, 25, 'DATE', '40150')
(28,

## Feature Extraction

In [70]:
# Worklist
# 1. Add types of perfume in the title

In [71]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['word'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['tag'].values.tolist())]
        self.grouped = self.data.groupby('sentence').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

        
getter = SentenceGetter(df)
sentences = getter.sentences

In [72]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [73]:
df

,sentence,word,pos,tag
0,0,已,D,nan
1,0,開封,VB,nan
2,0,clausporto,FW,nan
3,0,10,Neu,nan
4,0,ml,Nf,nan
...,...,...,...,...
9,827,已,D,nan
10,827,售出,VC,nan
11,827,1,Neu,nan
12,827,臺中市,Nc,nan


In [ ]:
X_pred = # cur_y為na

In [51]:
### Split train and test sets
X = [sent2features(s) for s in sentences]
new_y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=0)

### Train a CRF model

In [52]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [53]:
trainer.set_params({
#     'c1': 1.0,   # coefficient for L1 penalty
#     'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [54]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [55]:
trainer.train(path_export + '/ckip_tagger_model.crfsuite')

In [56]:
trainer.logparser.last_iteration

{'num': 20,
 'scores': {},
 'loss': 8.596126,
 'feature_norm': 2.10333,
 'error_norm': 1.6e-05,
 'active_features': 358,
 'linesearch_trials': 1,
 'linesearch_step': 1.0,
 'time': 0.0}

In [57]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

20 {'num': 20, 'scores': {}, 'loss': 8.596126, 'feature_norm': 2.10333, 'error_norm': 1.6e-05, 'active_features': 358, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.0}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [58]:
tagger = pycrfsuite.Tagger()
tagger.open(path_export + '/ckip_tagger_model.crfsuite')

Let's tag a sentence to see how it works:

In [59]:
y_pred = []
for i in range(len(y_test)):
    new_pred = tagger.tag(X_test[i])
    y_pred.append(new_pred)
#     example_sent = X_test[i]
#     print("Predicted:", ' '.join(tagger.tag(X_test[i])))    

In [60]:
y_pred = cbyz.li_flatten(y_pred)
y_test = cbyz.li_flatten(y_test)

In [61]:
print(classification_report(y_pred=y_pred, y_true=y_test))
# print(classification_report(y_pred=y_pred, y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        45
           1       1.00      1.00      1.00         2

    accuracy                           1.00        47
   macro avg       1.00      1.00      1.00        47
weighted avg       1.00      1.00      1.00        47



## Let's check what classifier learned

In [ ]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

## Word2Vec

In [ ]:
import gensim
from gensim import matutils
from gensim.models.word2vec import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format(
        path_resource + '/tmunlp_1.6B_WB_50dim_2020v1.bin.gz', 
        unicode_errors='ignore',
        binary=True
    )

In [ ]:
# Method 1

# wv_data = word_sentence_list
wv_data = word_sentence_list[0]

result = pd.DataFrame()

for i in range(len(wv_data)):

    word_vec = {}
    for w in word_sentence_list[i]:
        try:
            word_vec[w] = model.get_vector(w)
        except:
            pass

    # Create DF
    keys = list(word_vec.keys())
    vec_df = pd.DataFrame({'word':keys,
                          'index':range(len(keys))})

    vec_df = cbyz.df_cross_join(vec_df, vec_df)
    vec_df = vec_df[vec_df['index_x']<vec_df['index_y']] \
            .reset_index(drop=True) 
    
    for j in range(len(vec_df)):
        vec1 = model.get_vector(vec_df.loc[j, 'word_x'])
        vec2 = model.get_vector(vec_df.loc[j, 'word_y'])
        similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
        vec_df.loc[j, 'similarity'] = similarity

    result = result.append(vec_df)
    
    if i % 500 == 0:
        print(i, '/', len(word_sentence_list))
    
result = result.reset_index(drop=True)

In [ ]:
# Method 2

wv_data = word_sentence_list
# wv_data = word_sentence_list[0]

result = pd.DataFrame()

for i in range(len(wv_data)):

    word_vec = {}
    for w in word_sentence_list[i]:
        try:
            word_vec[w] = model.get_vector(w)
        except:
            pass

    # Create DF
    keys = list(word_vec.keys())
    vec_df = pd.DataFrame({'word':keys,
                          'index':range(len(keys))})

    for j in range(1, len(vec_df)):
        vec1 = model.get_vector(vec_df.loc[j, 'word'])
        vec2 = model.get_vector(vec_df.loc[j-1, 'word'])
        similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
        vec_df.loc[j, 'similarity'] = similarity

    result = result.append(vec_df)
    
    if i % 500 == 0:
        print(i, '/', len(word_sentence_list))
    
result = result.reset_index(drop=True)

In [ ]:
word_sentence_list[0]

In [ ]:
result.to_excel(path_export + '/wv_result.xlsx', index=False, encoding='utf-8-sig')

In [ ]:
寶格麗
馨香
浪漫
玫香
淡香精

bvlgari
寶格麗
晶澈
女性
淡香水


In [ ]:
result

In [ ]:
# https://en.wikipedia.org/wiki/Unicode_block
# Emoticons 

# import re
# regex = '\u3000-\u303F|\u2000-\u206F'
# regex = '\u2000-\u206F'
# regex = '\u4E00-\u9FA5|\u2000-\u206F'

# string = 'A臺中市->'
# # filtrate = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
# filtrate = re.compile(u'[^(' + regex + ')]') # non-Chinese unicode range
# string = filtrate.sub(r'', string) # remove all non-Chinese characters

# string

In [ ]:
result[result['similarity']>0.5]

In [ ]:
type(model)

In [ ]:
# model.vectors
# model.vocab

In [ ]:
len(model.vocab)

## Worklist

In [ ]:
# - 跟淡香水之間的距離
#   和男性、女性、中性的距離
# - 如果title中同時出現「淡香水」、「淡香精」就排除